<a href="https://colab.research.google.com/github/ask404/dish_RAG/blob/main/DishRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Recipe Dataset**
Source: Kaggle.com

In [ ]:
### Upload 3K recipes from Kaggle ###

# !pip install kaggle pandas
files.upload()  # Upload kaggle.json
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download crispen5gar/recipes3k
!unzip recipes3k.zip

Saving kaggle.json to kaggle (2).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/crispen5gar/recipes3k
License(s): GNU Free Documentation License 1.3
recipes3k.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  recipes3k.zip
replace baking.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: baking.json             
  inflating: budget.json             
  inflating: health.json             
  inflating: inspiration.json        
  inflating: recipes.json            


In [ ]:
import pandas as pd
import json

from google.colab import files

In [ ]:
# Load data
df = pd.read_json('recipes.json')  # or pd.read_json()

In [ ]:
df.columns

NameError: name 'filtered_df' is not defined

In [ ]:
df = df.rename(columns={
    'rattings': 'ratings'
})
df.columns

Index(['id', 'url', 'image', 'name', 'description', 'author', 'ratings',
       'ingredients', 'steps', 'nutrients', 'times', 'serves', 'difficult',
       'vote_count', 'subcategory', 'dish_type', 'maincategory'],
      dtype='object')

In [ ]:
df.head(5)

,id,url,image,name,description,author,ratings,ingredients,steps,nutrients,times,serves,difficult,vote_count,subcategory,dish_type,maincategory
0,7bac6ec3-fb66-4543-80f5-4b692de7a811,https://www.bbcgoodfood.com/recipes/smoked-sal...,https://images.immediate.co.uk/production/vola...,"Smoked salmon, quinoa & dill lunch pot",This easy packed lunch is as delicious as it i...,Chelsie Collins,4,"[2 tbsp half-fat soured cream, 2 tbsp lemon ju...","[First, make the dressing. Mix the soured crea...","{'kcal': '254', 'fat': '7g', 'saturates': '2g'...","{'Preparation': '15 mins', 'Cooking': 'No Time'}",2,Easy,10,Lunch recipes,Quick lunch recipes,recipes
1,25009e0a-0e60-4169-9f76-f464225549a9,https://www.bbcgoodfood.com/recipes/air-fryer-...,https://images.immediate.co.uk/production/vola...,Air fryer chicken breasts,Use an air fryer to create this tempting dish ...,Samuel Goldsmith,5,"[4 chicken breasts, ½ tbsp rapeseed oil, 1 ts...",[Coat the chicken in the oil and set aside. In...,{},"{'Preparation': '5 mins', 'Cooking': '18 mins ...",4,Easy,10,Lunch recipes,Quick lunch recipes,recipes
2,63ff8ff6-2871-4e92-96fd-39964c423a39,https://www.bbcgoodfood.com/recipes/panuozzo-s...,https://images.immediate.co.uk/production/vola...,Panuozzo sandwich,Make your own baguettes and pesto to make thes...,Paul Ainsworth,5,"[300g strong white bread flour, plus extra for...","[Put the flour in a large bowl, then stir in t...",{},"{'Preparation': '20 mins', 'Cooking': '12 mins'}",4,Easy,4,Lunch recipes,Quick lunch recipes,recipes
3,9e62330f-5ea0-4c2a-b0c8-edf386b63fb7,https://www.bbcgoodfood.com/recipes/bulgur-qui...,https://images.immediate.co.uk/production/vola...,Bulgur & quinoa lunch bowls,These meal prep grain bowls use one base and t...,Sara Buenfeld,5,"[1 large onion, very finely chopped, 150g bulg...","[Tip the onion and bulgur mix into a pan, pour...","{'kcal': '369', 'fat': '20g', 'saturates': '4g...","{'Preparation': '5 mins', 'Cooking': '15 mins'}",4,Easy,18,Lunch recipes,Quick lunch recipes,recipes
4,c095701c-a11e-4f47-90a0-a3ea20f7c752,https://www.bbcgoodfood.com/recipes/falafel-bu...,https://images.immediate.co.uk/production/vola...,Falafel burgers,A healthy burger that's filling too. These are...,Good Food team,4,"[400g can chickpeas, rinsed and drained, 1 sm...",[Drain the chickpeas and pat dry with kitchen ...,"{'kcal': '161', 'fat': '8g', 'saturates': '1g'...","{'Preparation': '10 mins', 'Cooking': '6 mins'}",4,Easy,710,Lunch recipes,Quick lunch recipes,recipes


In [ ]:
# If 'tags' doesn't exist, you can create it as an empty list/field
if 'tags' not in df.columns:
    df['tags'] = [[] for _ in range(len(df))]

In [ ]:
# Basic cleaning
def normalize_ingredients(text):
    # Handle both string representations of lists and actual lists:
    if isinstance(text, str):  # Check if 'text' is a string
        try:
            ingredients_list = json.loads(text)  # Try parsing as JSON if it's a string
        except json.JSONDecodeError:
            ingredients_list = eval(text)  # If JSON fails, attempt eval if it's a valid Python expression
    else:
        ingredients_list = text  # If 'text' is already a list, just use it

    return [ing.strip().lower() for ing in ingredients_list]

df['ingredients'] = df['ingredients'].apply(normalize_ingredients)

In [ ]:
# Add vegan tag
# def is_vegan(ingredients_list):
#     # A simplistic check: if none of certain keywords (like 'meat', 'egg', 'milk') appear
#     # This is not comprehensive but just an example:
#     non_vegan_keywords = ['meat', 'egg', 'milk', 'chicken',
#                           'pork', 'beef', 'fish', 'cheese', 'butter', 'honey']
#     for ing in ingredients_list:
#         for kw in non_vegan_keywords:
#             if kw in ing:
#                 return False
#     return True

# df['is_vegan'] = df['ingredients'].apply(is_vegan)
# df['tags'] = df.apply(lambda row: row['tags'] + ['vegan'] if row['is_vegan'] else row['tags'], axis=1)




In [ ]:
# Add vegetarian, gluten free, nut free and baby-friendly
diet_categories = {
    'vegan': [
        'meat', 'chicken', 'pork', 'beef', 'fish', 'seafood', 'shrimp',
        'lobster', 'bacon', 'ham', 'turkey', 'lamb', 'veal', 'venison',
        'gelatin', 'lard', 'animal broth', 'stock', 'bovine', 'pepsin',
        'egg', 'milk', 'cheese', 'butter', 'honey'
    ],
    'vegetarian': [
        'meat', 'chicken', 'pork', 'beef', 'fish', 'seafood', 'shrimp',
        'lobster', 'bacon', 'ham', 'turkey', 'lamb', 'veal', 'venison',
        'gelatin', 'lard', 'animal broth', 'stock', 'bovine', 'pepsin'
    ],
    'gluten_free': [
        'wheat', 'barley', 'rye', 'malt', 'triticale', 'brewer\'s yeast',
        'spelt', 'farro', 'semolina', 'couscous', 'seitan', 'durum',
        'bulgur', 'wheat flour', 'breadcrumbs', 'pasta', 'graham',
        'matzo', 'emmer', 'einkorn', 'fu', 'udon', 'soy sauce', 'teriyaki'
    ],
    'dairy_free': [
        'milk', 'cheese', 'butter', 'yogurt', 'cream', 'lactose',
        'casein', 'whey', 'ghee', 'curds', 'buttermilk', 'sour cream',
        'ice cream', 'custard', 'pudding', 'lactoglobulin', 'lactalbumin',
        'rennet', 'galactose', 'sodium caseinate', 'condensed milk'
    ],
    'nut_free': [
        'almond', 'cashew', 'walnut', 'pistachio', 'pecan', 'macadamia',
        'hazelnut', 'pine nut', 'brazil nut', 'chestnut', 'peanut',
        'nutella', 'praline', 'marzipan', 'nougat', 'nut oil', 'nut meal',
        'nut extract', 'beechnut', 'gianduja', 'mandelonas', 'nut paste'
    ],
    'baby_friendly': [
        'honey', 'alcohol', 'caffeine', 'raw egg', 'shellfish',
        'shark', 'swordfish', 'tilefish', 'king mackerel', 'added sugar',
        'monosodium glutamate', 'saccharin', 'sucralose', 'aspartame',
        'nitrate', 'nitrite', 'high-fructose', 'artificial color',
        'sodium benzoate', 'sulfites', 'whole nut', 'popcorn',
        'chili pepper', 'whole nuts', 'excess salt', 'coffee', 'tea'
    ]
}

def check_diet(ingredients_list, diet_keywords):
    for ing in ingredients_list:
        for kw in diet_keywords:
            if kw in ing.lower():
                return False
    return True

# Create diet category columns
for diet in diet_categories:
    df[f'is_{diet}'] = df['ingredients'].apply(
        lambda x: check_diet(x, diet_categories[diet])
    )

# Add tags
def update_tags(row):
    for diet in diet_categories:
        if row[f'is_{diet}']:
            row['tags'].append(diet.replace('_', '-'))
    return row

df = df.apply(update_tags, axis=1)

# Optional: Create combined dietary tags column
# df['dietary_tags'] = df.apply(
#     lambda row: [d.replace('_', '-') for d in diet_categories if row[f'is_{d}']],
#     axis=1
# )

In [ ]:
import re
def parse_time_string(time_str):
    """
    Parse time strings like '15 mins', '1 hr 15 mins', 'No Time' into minutes

    Args:
        time_str (str): Time string to parse

    Returns:
        float: Time in minutes, 0 for 'No Time'
    """
    if pd.isna(time_str) or time_str == 'No Time':
        return 0

    # Extract hours and minutes using regex
    hours = re.search(r'(\d+)\s*hr', time_str)
    minutes = re.search(r'(\d+)\s*mins?', time_str)

    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if minutes:
        total_minutes += int(minutes.group(1))

    return total_minutes

def parse_time_column(df, time_col='times'):
    """
    Parse the time dictionary column into separate numerical columns

    Args:
        df (pd.DataFrame): Input DataFrame
        time_col (str): Name of the column containing time dictionaries

    Returns:
        pd.DataFrame: DataFrame with added preparation_mins and cooking_mins columns
    """
    # Create a copy to avoid modifying the original
    df = df.copy()

    # Convert string dictionaries to actual dictionaries if needed
    if isinstance(df[time_col].iloc[0], str):
        df[time_col] = df[time_col].apply(eval)

    # Extract and parse times
    df['preparation_mins'] = df[time_col].apply(
        lambda x: parse_time_string(x.get('Preparation', '0 mins'))
    )
    df['cooking_mins'] = df[time_col].apply(
        lambda x: parse_time_string(x.get('Cooking', '0 mins'))
    )

    # Add total time column
    df['total_time_mins'] = df['preparation_mins'] + df['cooking_mins']

    return df

In [ ]:
df = parse_time_column(df)

NameError: name 'parse_time_column' is not defined

In [ ]:
## Filter recipes

def filter_recipes(df, max_total_time=None, min_rating=None, min_votes=None):
    """
    Filter recipes based on multiple criteria

    Args:
        df (pd.DataFrame): Input DataFrame
        max_total_time (float): Maximum total time in minutes
        min_rating (float): Minimum rating value
        min_votes (int): Minimum number of votes

    Returns:
        pd.DataFrame: Filtered DataFrame
    """
    filtered_df = df.copy()

    if max_total_time is not None:
        filtered_df = filtered_df[filtered_df['total_time_mins'] <= max_total_time]

    if min_rating is not None:
        filtered_df = filtered_df[filtered_df['ratings'] >= min_rating]

    if min_votes is not None:
        filtered_df = filtered_df[filtered_df['vote_count'] >= min_votes]

    return filtered_df


In [ ]:
filtered_df = filter_recipes(
    df,
    max_total_time=60,  # Max 1 hour total time
    min_rating=4.0,     # Minimum 4.0 rating
    min_votes=20        # Minimum 20 votes
)

In [ ]:
filtered_df.head(5)
# Sort by frequency (default is descending)
# filtered_df['dish_type'].value_counts().sort_values(ascending=False)

# Sort by frequency (default is descending)
filtered_df['subcategory'].value_counts().sort_values(ascending=False)


,count
subcategory,
Dinner recipes,50
Lunch recipes,41
Pasta,41
Vegan,39
Breakfast recipes,35
Fish and seafood,32
Storecupboard,31
Vegetarian,31
Desserts,28


In [ ]:
pd.set_option('display.max_colwidth', 400)

In [ ]:
df.head(1).transpose()

,0
id,7bac6ec3-fb66-4543-80f5-4b692de7a811
url,https://www.bbcgoodfood.com/recipes/smoked-salmon-quinoa-dill-lunch-pot
image,https://images.immediate.co.uk/production/volatile/sites/30/2020/08/smoked-salmon-quinoa-dill-lunch-pot-0393a04.jpg
name,"Smoked salmon, quinoa & dill lunch pot"
description,"This easy packed lunch is as delicious as it is nutritious, with crunchy cucumber and radishes and a herby, creamy dressing"
author,Chelsie Collins
ratings,4
ingredients,"[2 tbsp half-fat soured cream, 2 tbsp lemon juice, ½ pack dill, finely chopped, 250g pouch ready-to-eat quinoa (we used merchant gourmet), ½ cucumber, halved and sliced, 4 radishes, finely sliced, 100g smoked salmon, torn into strips]"
steps,"[First, make the dressing. Mix the soured cream and lemon juice together in a bowl, then add most of the dill, reserving a quarter for serving., In another bowl, combine the quinoa with the cucumber and radishes, and stir through half the dressing. Season and top with the salmon and the rest of the dill., Put the other half of the dressing in a small pot and drizzle over the quinoa just before..."
nutrients,"{'kcal': '254', 'fat': '7g', 'saturates': '2g', 'carbs': '26g', 'sugars': '3g', 'fibre': '5g', 'protein': '20g', 'salt': '2.5g'}"


In [ ]:
filtered_df.iloc[0].to_json()
#filtered_df[0][0]

'{"id":"c095701c-a11e-4f47-90a0-a3ea20f7c752","url":"https:\\/\\/www.bbcgoodfood.com\\/recipes\\/falafel-burgers-0","image":"https:\\/\\/images.immediate.co.uk\\/production\\/volatile\\/sites\\/30\\/2020\\/08\\/recipe-image-legacy-id-326597_11-b7385b9.jpg","name":"Falafel burgers","description":"A healthy burger that\'s filling too. These are great for anyone after a satisfying bite low in calories.","author":"Good Food team","ratings":4,"ingredients":["400g can  chickpeas, rinsed and drained","1 small red onion, roughly chopped","1 garlic clove, chopped","handful of flat-leaf parsley  or curly parsley","1 tsp ground cumin","1 tsp ground coriander","\\u00bd tsp harissa paste  or chilli powder","2 tbsp plain flour","2 tbsp sunflower oil","toasted pitta bread, to serve","200g tub tomato salsa, to serve","green salad, to serve"],"steps":["Drain the chickpeas and pat dry with kitchen paper. Tip into a food processor along with the onion, garlic, parsley, cumin, coriander, harissa paste, fl

In [ ]:
import pandas as pd


def display_entries(df, **kwargs):
    """
    Display specific entries in a DataFrame based on various filtering criteria.

    Parameters:
    df (pandas.DataFrame): Input DataFrame
    **kwargs: Filtering criteria
        - columns: List of column names to display
        - row_indices: List of row indices to display
        - conditions: Dict of column:value pairs to filter by
            For single values: {'column': value}
            For list of values: {'column': [value1, value2, ...]}
        - head: Number of first rows to display
        - tail: Number of last rows to display

    Returns:
    pandas.DataFrame: Filtered DataFrame
    """
    result = df.copy()

    # Filter columns if specified
    if 'columns' in kwargs:
        result = result[kwargs['columns']]

    # Filter by row indices
    if 'row_indices' in kwargs:
        result = result.iloc[kwargs['row_indices']]

    # Filter by conditions
    if 'conditions' in kwargs:
        for column, value in kwargs['conditions'].items():
            if isinstance(value, (list, tuple, set)):
                # If value is a list, use isin() for filtering
                result = result[result[column].isin(value)]
            else:
                # For single values, use equality
                result = result[result[column] == value]

    # Get head or tail
    if 'head' in kwargs:
        result = result.head(kwargs['head'])
    elif 'tail' in kwargs:
        result = result.tail(kwargs['tail'])

    return result

In [ ]:
display_entries(filtered_df,
                conditions={'id': id_list }
                #,columns=['name']
                #,'description','steps','subcategory', 'ratings', 'total_time_mins']
                )

NameError: name 'filtered_df' is not defined

In [ ]:
# Example 1: Filter by a single value condition
# Show all entries where age is 30
# result1 = display_entries(df, conditions={'age': 30})
# print(result1)
# Output:
#    name  age    city  salary
# 1  Alice  30  Boston   60000
# 3  Carol  30  Boston   65000

# Example 2: Filter by a list of values
# Show entries where city is either Boston or Chicago
# result2 = display_entries(df,
#                          conditions={'city': ['Boston', 'Chicago']},
#                          columns=['name', 'city', 'salary'])
# print(result2)


display_entries(filtered_df,
                conditions={'name': 'Kale Pesto Toast with Roasted Chickpeas'}
)

# **Vector Database Setup**
Using: Pinecone

In [ ]:
## Install Pinecone in Colab
!pip install pinecone-client
!pip install openai


In [ ]:
## INITIALIZE OPENAI

from google.colab import userdata

# Embed Each Recipe
import openai
from openai import OpenAI # Import the OpenAI class

openai_key = userdata.get('openai_key')
# Instead of setting openai.api_key, create an OpenAI client instance:
client = OpenAI(api_key=openai_key)


In [ ]:
## DEFINE RECIPE EMBEDDING FUNCTION

def get_recipe_embedding(text):
    # Use the client to create embeddings:
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    # The API returns a list of embeddings; we take the first one
    embedding = response.data[0].embedding  # Access using .data[0].embedding
    return embedding

# You can combine title, ingredients, and instructions:
def combine_recipe_text(row):
    name = row['name']
    description = row['description']
    ingredients = " ".join(row['ingredients'])  # or keep as comma separated
    steps = row['steps']
    return f"{name}\n\nDescription:\n{description}\n\nIngredients:\n{ingredients}\n\nInstructions:\n{steps}"

In [ ]:
filtered_df['combined_text'] = filtered_df.apply(combine_recipe_text, axis=1)

In [ ]:
## Compute embeddings for each recipe.
embeddings = []
for idx, row in filtered_df.iterrows():
    text = row['combined_text']
    embedding = get_recipe_embedding(text)
    embeddings.append(embedding)

# Add to the dataframe
filtered_df['embedding'] = embeddings

In [ ]:
## CREATE A NEW PINECONE INDEX
import pinecone
from pinecone import ServerlessSpec

pinecone_key = userdata.get('pinecone_key')
pinecone_client = pinecone.Pinecone(api_key=pinecone_key)
index_name = "recipe-index"

# Check if index exists
if index_name in pinecone_client.list_indexes().names():
    confirmation = input(f"Warning: Index '{index_name}' already exists. Type 'YES' to delete and recreate: ")
    if confirmation == "YES":
        pinecone_client.delete_index(index_name)
        print(f"Deleted existing index: {index_name}")
    else:
        raise Exception("Operation cancelled by user")

# Create new index
pinecone_client.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ),
    deletion_protection="disabled"
)
print(f"Created new index: {index_name}")

# Connect to the new index
index = pinecone_client.Index(index_name)

KeyboardInterrupt: Interrupted by user

In [ ]:
# PREPARE DATA FOR UPSERT INTO PINECONE INDEX
index_name = "recipe-index"
# Connect to the new index
index = pinecone_client.Index(index_name)
"""
Pinecone’s upsert expects a list of tuples in the form (id, vector, metadata) where:
- id is a unique string (e.g., the row index or a recipe ID).
- vector is the embedding list (dimension = 1536).
- metadata is a Python dictionary containing any other info you want to store.
"""

data_to_upsert = []
for idx, row in filtered_df.iterrows():
    recipe_id = str(idx)  # or use your own unique ID
    vector = row['embedding']
    metadata = {
        'recipe_id': row['id'],
        'name': row['name'],
        'tags': row['tags'],
        'is_vegan': row['is_vegan'] if 'is_vegan' in row else False,
        'is_vegetarian': row['is_vegetarian'] if 'is_vegetarian' in row else False,
        'is_gluten_free': row['is_gluten_free'] if 'is_gluten_free' in row else False,
        'is_nut_free': row['is_nut_free'] if 'is_nut_free' in row else False,
        'is_dairy_free': row['is_dairy_free'] if 'is_dairy_free' in row else False,
        'is_baby_friendly': row['is_baby_friendly'] if 'is_baby_friendly' in row else False,
        'ratings': row['ratings'],
        'cuisine': row['subcategory'],
        'difficulty': row['difficult']
        # add anything else you want, e.g. cuisine type...
    }
    data_to_upsert.append((recipe_id, vector, metadata))

# Because it's more efficient, upsert in batches
batch_size = 100
for i in range(0, len(data_to_upsert), batch_size):
    batch = data_to_upsert[i:i+batch_size]
    index.upsert(vectors=batch)



# Upload recipes into Firestore

In [ ]:
## FIRESTORE: Upload Recipes into Firestore
### FUTURE OPTIMIZTAION -- do NOT upload the embeddings (so before they are added to the Dataframe or filter them out after the fact)
# 1) Convert DF to list of dicts
recipes_list = filtered_df.to_dict(orient='records')

# 2) Upload to Firestore
collection_name = "recipes"
batch_count = 0

for recipe_dict in recipes_list:
    # Use the recipe's "id" as the Firestore doc ID
    doc_id = recipe_dict["id"]
    doc_ref = db.collection(collection_name).document(doc_id)
    doc_ref.set(recipe_dict)
    batch_count += 1

print(f"Uploaded {batch_count} recipes to the '{collection_name}' collection.")

Uploaded 390 recipes to the 'recipes' collection.


# **Run Existing Program**

In [13]:
## INITIALIZE EXISTING PINECONE INDEX AND OPENAI API
!pip install pinecone-client
!pip install openai

from google.colab import userdata

import pinecone
from pinecone import ServerlessSpec

pinecone_key = userdata.get('pinecone_key')
pinecone_client = pinecone.Pinecone(api_key=pinecone_key)
index_name = "recipe-index"

import openai
from openai import OpenAI # Import the OpenAI class

openai_key = userdata.get('openai_key')
# Instead of setting openai.api_key, create an OpenAI client instance:
client = OpenAI(api_key=openai_key)

# Connect to the new index
index = pinecone_client.Index(index_name)
# Verify Data
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 390}},
 'total_vector_count': 390}

In [14]:
# Define get recipe embedding function

def get_recipe_embedding(text):
    # Use the client to create embeddings:
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    # The API returns a list of embeddings; we take the first one
    embedding = response.data[0].embedding  # Access using .data[0].embedding
    return embedding

In [15]:
# Test Similarity Queries: You can use Pinecone’s query endpoint.
test_text = "breakfast dish with eggs cheese carrots and broccoli"
test_vector = get_recipe_embedding(test_text)

In [16]:
result = index.query(vector=test_vector, top_k=5, include_metadata=True)
## Extract ids from index
def extract_recipe_ids(data):
    # Extract recipe_ids from the metadata of each match
    recipe_ids = [match['metadata']['recipe_id'] for match in data['matches']]
    return recipe_ids
id_list = extract_recipe_ids(result)

print(result)

{'matches': [{'id': '200',
              'metadata': {'cuisine': 'Breakfast recipes',
                           'difficulty': 'Easy',
                           'is_baby_friendly': True,
                           'is_dairy_free': False,
                           'is_gluten_free': False,
                           'is_nut_free': True,
                           'is_vegan': False,
                           'is_vegetarian': True,
                           'name': 'Easy egg muffins',
                           'ratings': 4.0,
                           'recipe_id': 'b3ec2c92-d959-4dde-b2b3-b7383c8f0037',
                           'tags': ['vegetarian', 'nut-free', 'baby-friendly']},
              'score': 0.843298674,
              'values': []},
             {'id': '53',
              'metadata': {'cuisine': 'Lunch recipes',
                           'difficulty': 'Easy',
                           'is_baby_friendly': True,
                           'is_dairy_free': False,
       

CREATE Retrieval Function

This function will:

1.   Take the user’s query (which might include desired ingredients, time constraints, or dietary flags).   
2.   Convert it into a single string to embed.

1.   Optionally parse out dietary requirements (e.g., vegan, baby-friendly) to use as Pinecone filters.

1.   Query Pinecone for the most similar recipes.

1.   Return the top K matching recipes (with metadata and text if needed).





In [17]:
def get_relevant_recipes(user_query, top_k=5):
    """
    user_query: e.g. "vegan dish with chickpeas under 30 minutes, also baby-friendly"
    top_k: number of recipes to retrieve
    """
    # STEP 1: Parse out filters if needed.
    #   This can be fancy NLP or simple keyword checks:
    is_vegan_filter = "vegan" in user_query.lower()
    is_baby_filter = "baby-friendly" in user_query.lower()
    is_vegetarian_filter = "vegetarian" in user_query.lower()
    is_gluten_free_filter = "gluten free" in user_query.lower()
    is_dairy_free_filter = "dairy free" in user_query.lower()
    is_nut_free_filter = "nut free" in user_query.lower()

    # STEP 2: Create a simple text query to embed
    query_embedding = get_recipe_embedding(user_query)

    # STEP 3: Build Pinecone metadata filter
    #   If is_vegan_filter is True, we want is_vegan == True
    #   If is_baby_filter is True, we want is_baby_friendly == True
    #   If not specified, we ignore that filter
    # Pinecone’s filter syntax is a dictionary of "field": value
    pinecone_filter = {}
    if is_vegan_filter:
        pinecone_filter["is_vegan"] = True
    if is_baby_filter:
        pinecone_filter["is_baby_friendly"] = True
    if is_nut_free_filter:
        pinecone_filter["is_nut_free"] = True
    if is_vegetarian_filter:
        pinecone_filter["is_vegetarian"] = True
    if is_gluten_free_filter:
        pinecone_filter["is_gluten_free"] = True
    if is_dairy_free_filter:
        pinecone_filter["is_dairy_free"] = True

    # STEP 4: Query the Pinecone index
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        filter=pinecone_filter if pinecone_filter else None
    )

    # STEP 5: Format the returned matches
    #   Each match has an 'id', 'score', and 'metadata' we can use
    recipes = []
    for match in results['matches']:
        metadata = match['metadata']
        recipe_info = {
            'id': match['id'],
            'score': match['score'],
            'name': metadata.get('name', ''),
            # 'description': metadata.get('description', ''),
            'is_vegan': metadata.get('is_vegan'),
            'is_baby_friendly': metadata.get('is_baby_friendly'),
            # 'ingredients':metadata.get('ingredients',''),
            # 'steps':metadata.get('steps',''),
            'tags':metadata.get('tags',''),
            'ratings':metadata.get('ratings',''),
            'total_time_mins':metadata.get('total_time_mins',''),
            'recipe_id': metadata.get('recipe_id')
        }
        recipes.append(recipe_info)

    return recipes

In [18]:
sample_query = "I have kale, chickpeas, bell peppers and sourdough bread.  Can you suggest a open-faced toast recipe?"
retrieved_recipes = get_relevant_recipes(sample_query, top_k=3)
retrieved_recipes

[{'id': '354',
  'score': 0.820189595,
  'name': 'Halloumi flatbreads',
  'is_vegan': True,
  'is_baby_friendly': True,
  'tags': ['vegan',
   'vegetarian',
   'gluten-free',
   'dairy-free',
   'nut-free',
   'baby-friendly'],
  'ratings': 5.0,
  'total_time_mins': '',
  'recipe_id': '6a01beb0-1b09-4d2e-ad6e-01c01ae8c6c6'},
 {'id': '37',
  'score': 0.805488408,
  'name': 'Feta & kale loaded sweet potato',
  'is_vegan': True,
  'is_baby_friendly': True,
  'tags': ['vegan',
   'vegetarian',
   'gluten-free',
   'dairy-free',
   'nut-free',
   'baby-friendly'],
  'ratings': 5.0,
  'total_time_mins': '',
  'recipe_id': 'b9bed024-48f8-45eb-8177-a2a35b1888a2'},
 {'id': '388',
  'score': 0.803791881,
  'name': 'Courgette, leek & goat’s cheese soup',
  'is_vegan': False,
  'is_baby_friendly': True,
  'tags': ['vegetarian', 'nut-free', 'baby-friendly'],
  'ratings': 4.0,
  'total_time_mins': '',
  'recipe_id': 'b0d04518-8700-4eda-9141-82e6fee6ed1a'}]

In [19]:
### Initialize Firestore

!pip install firebase-admin
from google.colab import files
import firebase_admin
from firebase_admin import credentials, firestore

# Upload the service account JSON file
uploaded = files.upload()

# Get the filename of the uploaded JSON
filename = list(uploaded.keys())[0]

# Initialize Firebase only once
if not firebase_admin._apps:
    cred = credentials.Certificate(filename)
    firebase_admin.initialize_app(cred)

# Get Firestore client
db = firestore.client()

Saving dishrag-abaa6-firebase-adminsdk-fbsvc-048df1ffdf.json to dishrag-abaa6-firebase-adminsdk-fbsvc-048df1ffdf (1).json


In [20]:
## FIRESTORE: Fetching a Recipe from Firestore
# Create a helper function to fetch recipe data by ID:

def get_recipe_from_db(recipe_id):
    """
    Fetch the recipe document from Firestore by recipe_id, retrieving only entries
    with 'embedding' field.
    Returns None if no document found.
    """
    doc_ref = db.collection("recipes").document(recipe_id)
    doc = doc_ref.get()

    if doc.exists:
        data = doc.to_dict()
        # Filter for all entries except for the 'embedding' field
        if 'embedding' in data:
            return {
                key: value for key, value in data.items()
                if key != 'embedding'
            }
    return None

In [21]:
print(get_recipe_from_db('b3ec2c92-d959-4dde-b2b3-b7383c8f0037'))

{'nutrients': {}, 'preparation_mins': 15, 'maincategory': 'recipes', 'is_vegan': False, 'is_vegetarian': True, 'subcategory': 'Breakfast recipes', 'name': 'Easy egg muffins', 'dish_type': 'Breakfast muffins', 'difficult': 'Easy', 'vote_count': 37, 'dietary_tags': ['vegetarian', 'nut-free', 'baby-friendly'], 'steps': ['Heat the oven to 200C/180C fan/gas 4. Brush half the oil in an 8-hole muffin tin. Heat the remaining oil in a frying pan and add the broccoli, pepper and spring onions. Fry for 5 mins. Set aside to cool. ', 'Whisk the eggs with the milk, smoked paprika and half the cheese in a bowl. Add the cooked veg. Pour the egg mixture into the muffin holes and top each with the remaining cheese and a few chives, if you like. Bake for 15-17 mins or until golden brown and cooked through. '], 'total_time_mins': 40, 'is_nut_free': True, 'times': {'Cooking': '25 mins', 'Preparation': '15 mins'}, 'description': 'Make these mini egg muffins for an easy breakfast or lunch with the kids. Add 

In [26]:
# Revised generate_recipe_suggestion with Firestore

# Below is your updated function. Notice how:

# We only rely on Pinecone to give us a recipe_id.
# We fetch full recipe details from Firestore.
# We build the prompt with the full recipe text, not stored in Pinecone.

import openai
def build_recipe_text_blocks(retrieved_recipes):
    """
    Takes retrieved recipes and builds formatted text blocks with full recipe data from Firestore.
    Returns a list of formatted recipe text blocks.
    """
    recipe_text_blocks = []

    for i, recipe in enumerate(retrieved_recipes, start=1):
        recipe_id = recipe['recipe_id']
        full_recipe = get_recipe_from_db(recipe_id)

        if not full_recipe:
            continue

        name = full_recipe.get("name", "Unknown Recipe")
        ingredients = full_recipe.get("ingredients", [])
        steps = full_recipe.get("steps", [])
        description = full_recipe.get("description", "")
        ratings = full_recipe.get("ratings", "N/A")
        vote_count = full_recipe.get("vote_count", "N/A")
        total_time_mins = full_recipe.get("total_time_mins", "N/A")
        serves = full_recipe.get("serves", "N/A")
        image = full_recipe.get("image", "N/A")
        recipe_type = full_recipe.get("subcategory", "N/A")
        difficulty = full_recipe.get("difficult", "N/A")
        recipe_id = full_recipe.get("id", "N/A")
        recipe_url = full_recipe.get("url", "N/A")

        block = (
            f"{i}) {name}\n"
            f"Description: {description}\n"
            f"Ingredients: {', '.join(ingredients)}\n"
            f"Steps: {' '.join(steps)}\n"
            f"Ratings: {ratings}\n"
            f"Total Time: {total_time_mins} minutes\n"
            f"Recipe ID: {recipe_id}\n"
            f"Vote Count: {vote_count}\n"
            f"Serving Size: {serves}\n"
            f"Image: {image}\n"
            f"Recipe URL: {recipe_url}\n"
            f"Recipe Type: {recipe_type}\n"
            f"Difficulty: {difficulty}\n"
        )
        recipe_text_blocks.append(block)

    return recipe_text_blocks

In [36]:

def generate_recipe_suggestion_with_db(retrieved_recipes, user_query):
    """
    Takes top retrieved recipes (with minimal metadata + ID from Pinecone)
    plus user query, fetches full recipe data from Firestore,
    crafts a prompt, calls GPT-3.5, returns the model's response.
    """
    recipe_text_blocks = build_recipe_text_blocks(retrieved_recipes)
    combined_recipe_text = "\n".join(recipe_text_blocks)

    prompt = f"""
    You are a helpful chef.
    The user wants: {user_query}.
    Here are {len(retrieved_recipes)} recipe options that match:
    {combined_recipe_text}
    Using ONLY the above information, please suggest the best recipe or a new creative combination based on these options.
    DO NOT MAKE UP RECIPES.
    Reprint the name, description, author, average user ratings, number of votes, ingredients (including measurements), steps, serving size, difficulty, recipe type, and total prep/cooking time of the suggested recipe.
    Explain why it's a good fit, and keep it concise.
    Only use the recipes provided for your answer.
    At the end print the name, description, author, average user ratings, recipe type, and number of votes of two other top ranked recipes and ask if the user would like to see the full recipe of either.
    At the bottom of your response print the recipe IDs of each of the recipes followed by the URL of each recipe referenced in your response.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful recipe assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.4,
        max_tokens=1000
    )

    answer = response.choices[0].message.content
    return answer

In [24]:
sample_query = "I have kale, chickpeas, bell peppers and sourdough bread.  Can you suggest a open-faced toast recipe?"
retrieved_recipes = get_relevant_recipes(sample_query, top_k=3)
retrieved_recipes
print(retrieved_recipes)

[{'id': '354', 'score': 0.820189595, 'name': 'Halloumi flatbreads', 'is_vegan': True, 'is_baby_friendly': True, 'tags': ['vegan', 'vegetarian', 'gluten-free', 'dairy-free', 'nut-free', 'baby-friendly'], 'ratings': 5.0, 'total_time_mins': '', 'recipe_id': '6a01beb0-1b09-4d2e-ad6e-01c01ae8c6c6'}, {'id': '37', 'score': 0.805488408, 'name': 'Feta & kale loaded sweet potato', 'is_vegan': True, 'is_baby_friendly': True, 'tags': ['vegan', 'vegetarian', 'gluten-free', 'dairy-free', 'nut-free', 'baby-friendly'], 'ratings': 5.0, 'total_time_mins': '', 'recipe_id': 'b9bed024-48f8-45eb-8177-a2a35b1888a2'}, {'id': '388', 'score': 0.803791881, 'name': 'Courgette, leek & goat’s cheese soup', 'is_vegan': False, 'is_baby_friendly': True, 'tags': ['vegetarian', 'nut-free', 'baby-friendly'], 'ratings': 4.0, 'total_time_mins': '', 'recipe_id': 'b0d04518-8700-4eda-9141-82e6fee6ed1a'}]


In [27]:
print(build_recipe_text_blocks(retrieved_recipes))

['1) Halloumi flatbreads\nDescription: Try these simple veggie halloumi wraps with a crunchy slaw and a generous helping of hummus. A great cheesy budget option for a midweek meal\nIngredients: 50g pumpkin seeds, 2 tbsp cumin seeds, 1 small red cabbage (about 650g), core removed, shredded, 2 mixed peppers, cut into strips, 2 tbsp white wine vinegar, 2 tbsp extra virgin olive oil, 6-8 flatbreads or wraps, 100g hummus, 80g bag rocket, 2 x 250g blocks halloumi, each cut into 12 strips\nSteps: Toast the pumpkin seeds with the cumin seeds in a large frying pan until they begin to pop and smell fragrant, then transfer to a large bowl. Add the cabbage, peppers and vinegar, and season well. Mix thoroughly and set aside. Heat oven to low and put the bread in to warm through. Using the pan you used to toast the seeds, fry the halloumi with the olive oil in batches for 3 mins each side until crispy and golden. Transfer to the oven to keep warm. Spread a layer of hummus on each flatbread and top w

In [37]:
# Example usage (assuming you already have retrieved_recipes from Pinecone):
test_answer = generate_recipe_suggestion_with_db(retrieved_recipes, sample_query)
print(test_answer)

The best recipe option based on your ingredients is **Feta & kale loaded sweet potato**. While it primarily features sweet potatoes, you can adapt it to create an open-faced toast by using your sourdough bread as the base. The combination of kale, chickpeas, and feta makes for a nutritious and flavorful topping.

### Feta & Kale Loaded Sweet Potato
**Description:** Serve up this sweet potato packed with feta, chickpeas, and kale for a healthy lunch. Topped with pumpkin seeds, it delivers three of your 5-a-day.  
**Author:** BBC Good Food  
**Average User Ratings:** 5  
**Number of Votes:** 24  

**Ingredients:**
- 2 sweet potatoes
- Chickpeas, drained
- 1 red onion, thinly sliced
- 2 tbsp red wine vinegar
- 30g feta, cut into small cubes
- 1 tsp caster sugar
- 1 tbsp olive oil
- Chilli flakes
- 100g kale
- 1 tbsp pumpkin seeds, toasted
- Rocket

**Steps:**
1. Heat oven to 200C/180C fan/gas 6. Prick the sweet potatoes all over with a fork, then put them in a roasting tin and roast for 4

# Gradio

In [ ]:
#import Gradio
!pip install gradio
import gradio as gr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
tor

In [ ]:
# !pip install gradio openai pinecone-client firebase-admin

# import gradio as gr
# import openai
# import pinecone
# import firebase_admin
# from firebase_admin import credentials, firestore

# openai.api_key = "YOUR_OPENAI_API_KEY"

# pinecone.init(
#     api_key="YOUR_PINECONE_API_KEY",
#     environment="YOUR_PINECONE_ENVIRONMENT"
# )
# YOUR_INDEX_NAME = "recipe-index"  # e.g., "recipe-index"
# index = pinecone.Index(YOUR_INDEX_NAME)


# cred = credentials.Certificate("path/to/your/serviceAccountKey.json")
# firebase_admin.initialize_app(cred)
# db = firestore.client()

# def get_relevant_recipes(user_query, top_k=3):
#     """
#     YOUR LOGIC to query Pinecone with user_query for top_k recipes.
#     Return a list of recipe dicts containing at least an 'id' to fetch Firestore doc.
#     """
#     # Example placeholder code
#     results = index.query(...)  # your Pinecone query
#     # build a list of recipes with IDs
#     return [ ... ]

# def get_recipe_from_db(recipe_id):
#     """
#     Fetch the recipe document from Firestore by recipe_id.
#     Returns a dict with full fields, or None if doc not found.
#     """
#     doc_ref = db.collection("recipes").document(recipe_id)
#     doc = doc_ref.get()
#     if doc.exists:
#         return doc.to_dict()
#     else:
#         return None

# def generate_recipe_suggestion_with_db(retrieved_recipes, user_query):
#     """
#     Takes top retrieved recipes + user_query, fetches full recipe data from Firestore,
#     constructs a prompt, calls an LLM, returns the LLM's text response.
#     """
#     # Your existing logic
#     return "LLM-generated suggestion..."


 ## Decide When to Retrieve vs. Continue the Conversation

 We want logic to distinguish a brand new request from a refinement. One simple approach is:

*   If the user’s message looks like a new request (e.g., contains new dietary requirements or user specifically says “I need a new recipe for ...”), run the Pinecone retrieval again.
*  Otherwise, just feed the conversation into the LLM (e.g., “Can I substitute X?”).

In [ ]:
def needs_new_retrieval(user_message, state=None):
    """
    Determine if a new retrieval is needed based on the message content and chat state.

    Args:
        user_message (str): The current user message
        state (dict, optional): Gradio chat state dictionary. If None, assumes this is first message.

    Returns:
        bool: True if new retrieval is needed, False otherwise
    """
    # In Gradio, chat history is stored in state. If no state or empty state,
    # this is the first message of the session
    if state is None or not state.get('messages', []):
        return True

    # Check for recipe-related keywords as a fallback
    trigger_keywords = [
        "recipe for",
        "new recipe",
        "ingredients for",
        "I need a dish",
        "I want a recipe"
    ]
    user_lower = user_message.lower()
    return any(keyword in user_lower for keyword in trigger_keywords)

# Define the Chat Handler

We’ll maintain:


*   A list of chat history messages: (user_message, bot_reply)
*   A list or variable with the last retrieved recipes (to avoid repeated Pinecone calls if the user is just refining).

In Gradio, you typically define a function that takes the user input and the current state, returns the updated conversation and state.

In [ ]:
def chat_handler(user_input, chat_history, last_retrieved_recipes):
    """
    user_input: current user query (string).
    chat_history: list of (user_message, bot_message) from previous turns.
    last_retrieved_recipes: what we stored after the last retrieval. Could be a list of recipes or None.

    Returns updated chat_history and updated last_retrieved_recipes.
    """

    # 1) Decide if we need a new retrieval
    if needs_new_retrieval(user_input):
        # This is a brand new query or major change
        top_k = 5  # or set dynamically
        retrieved = get_relevant_recipes(user_input, top_k=top_k)

        # Then generate LLM response based on these new recipes
        llm_response = generate_recipe_suggestion_with_db(retrieved, user_input)

        # Update last_retrieved_recipes with the new retrieval
        last_retrieved_recipes = retrieved
    else:
        # This is a refinement; just feed the conversation to GPT
        # Optionally pass last_retrieved_recipes if needed
        # E.g., you could build a simpler prompt like "User asks: {user_input}. The last recipes: {last_retrieved_recipes}"
        # Or directly feed the entire chat_history to the LLM. For demonstration:

        llm_response = refine_existing_conversation(chat_history, user_input, last_retrieved_recipes)
        # ^ define "refine_existing_conversation" to call GPT with the entire context
        #   or just do a standard ChatCompletion with the conversation. This is up to you.

    # 2) Add the user and bot messages to the chat history
    chat_history.append((user_input, llm_response))

    return chat_history, last_retrieved_recipes

In [ ]:
## About refine_existing_conversation
# Implement a function that calls your LLM with the entire chat context plus the user’s new question.
# If your conversation gets too long, consider summarizing older messages.


def refine_existing_conversation(chat_history, user_input, last_retrieved_recipes):
    """
    Called when the user is *refining* the conversation instead of requesting a brand-new query.
    We incorporate the previous chat_history and the last_retrieved_recipes to provide context.
    """
    messages = []

    # SYSTEM INSTRUCTION:
    messages.append({"role": "system", "content": "You are a helpful recipe assistant."})

    # PRIOR TURNS: user/assistant pairs from chat_history
    for user_msg, bot_msg in chat_history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})

    # INCORPORATE PREVIOUSLY RETRIEVED RECIPES (if any)
    if last_retrieved_recipes:
        # Build a short summary of the last retrieved recipes
        # (You can customize this to include important fields like name, short description, IDs, etc.)
        recipes_reminder = "Here are the recipes we found previously:\n"
        for i, recipe in enumerate(last_retrieved_recipes, start=1):
            recipes_reminder += (
                f"{i}) {recipe.get('name', 'Unnamed Recipe')} (ID: {recipe.get('id', 'N/A')})\n"
                f"   Description: {recipe.get('description', '')[:100]}...\n"
            )

        # We can either append this as an additional user message OR inject it into the user's final prompt.
        # Let's append it as an additional "assistant" message so the LLM sees it as context:
        messages.append({"role": "assistant", "content": recipes_reminder})

    # CURRENT USER MESSAGE: what the user just typed
    # Optionally, you might combine "recipes_reminder" directly into the user_input.
    # But here we keep them separate for clarity.
    messages.append({"role": "user", "content": user_input})

    # CALL THE LLM (replace with your actual LLM client)
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or "gpt-3.5-turbo", etc.
        messages=messages,
        temperature=0.6,
        max_tokens=650
    )

    answer = response.choices[0].message.content
    return answer


# Handling Images in Gradio

If each recipe has an image URL (e.g., image="https://..."), you can display it in Gradio.

*   After generating the suggestion, extract the chosen recipe’s image URL from Firestore or from the LLM response if it’s included.
*   Create a small function that downloads or returns that image for Gradio. You can set the .value of a gr.Image component.



In [ ]:
import requests
from PIL import Image
import io

def fetch_image_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        img = Image.open(io.BytesIO(response.content))
        return img
    return None

def update_recipe_image(recipe_image_url):
    img = fetch_image_from_url(recipe_image_url)
    return img

# Build the Gradio Interface

We’ll use Gradio Blocks for flexibility. We’ll store two pieces of state:

*   chat_history: a list of (user_msg, bot_msg) tuples for the conversation.
*   last_retrieved_recipes: the most recent Pinecone retrieval results (list of recipes or None).


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Recipe Chat Assistant")

    # A chatbot component to display conversation
    chatbot = gr.Chatbot(label="Chat with the Recipe Assistant", height=400)
    state = gr.State(value={"messages": []})

    # State variables
    state_chat_history = gr.State([])          # List of (user, bot) messages
    state_retrieved = gr.State(None)           # Last Pinecone retrieval results

    # A row with user input and submit
    with gr.Row():
        user_input = gr.Textbox(show_label=False, placeholder="Ask for a recipe...", lines=2)
        submit_btn = gr.Button("Send")

    # Define the function that ties it all together
    def respond_to_user(user_input, chat_history, last_retrieved):
        updated_chat_history, updated_retrieved = chat_handler(user_input, chat_history, last_retrieved)
        # Return the new chat history to display, and store in state
        return updated_chat_history, updated_chat_history, updated_retrieved

    # Link the submit button to the function
    submit_btn.click(
        fn=respond_to_user,
        inputs=[user_input, state_chat_history, state_retrieved],
        outputs=[chatbot, state_chat_history, state_retrieved]
    )

    # Optionally, show an image or other dynamic UI
    # For example, a separate area to display the main recommended recipe image
    recipe_image = gr.Image(label="Recipe Image (if available)")

    # In a more advanced scenario, you'd update this image after each LLM suggestion
    # e.g., parse the chosen recipe's 'image' URL, load it, and display.
    # That might require a second function or an event to set the image.

demo.launch(debug=True)


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://92d5e16d7a6d9c5d88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://92d5e16d7a6d9c5d88.gradio.live


In [ ]:
#define preset ingredients and dietary options

common_ingredients = ["Eggs", "Milk", "Flour", "Sugar", "Butter",
                      "Chicken", "Tomatoes", "Onions", "Garlic", "Cheese"]
diet_options = ["Vegetarian", "Vegan", "Gluten-Free", "Dairy-Free", "Nut-Free", "Baby-Friendly"]


In [ ]:
#create UI

with gr.Blocks() as app:
    gr.Markdown("# 🍳 DishRAG")
    gr.Markdown("Select ingredients and dietary restrictions to find recipe ideas:")

    # 1. Ingredient Selection: multi-select dropdown (pillbox style)
    ingredient_selector = gr.Dropdown(
        choices=common_ingredients,
        label="Select Ingredients",
        multiselect=True,
        info="Choose one or more ingredients you want to use"
    )

    # 2. Dietary Restrictions: multiple selection via checkboxes
    diet_selector = gr.CheckboxGroup(
        choices=diet_options,
        label="Dietary Restrictions",
        info="Select any dietary requirements"
    )

    # Search button to trigger recipe lookup
    find_button = gr.Button("Find Recipes")

    # 3. Suggestions Section
    with gr.Group(visible=True) as suggestions_section:
        # Navigation + view details controls
        with gr.Row():
            prev_btn = gr.Button("⬅️ Previous")
            view_btn = gr.Button("📄 View Details")
            next_btn = gr.Button("Next ➡️")
        # Current recipe card display
        recipe_card = gr.Markdown("")  # will show recipe name (and maybe rating)

    # 4. Detail Section (initially hidden)
    with gr.Group(visible=False) as detail_section:
        back_btn = gr.Button("🔙 Back to Results")
        recipe_details_md = gr.Markdown("")

    # State variables to store suggestions list and current index
    suggestions_state = gr.State([])  # list of suggestion dicts
    current_index = gr.State(0)

    # Function to fetch suggestions based on inputs
    diet_flags_mapping = {
        "Vegan": "is_vegan",
        "Vegetarian": "is_vegetarian",
        "Gluten-Free": "is_gluten_free",
        "Dairy-Free": "is_dairy_free",
        "Nut-Free": "is_nut_free",
        "Baby-Friendly": "is_baby_friendly",
        # extend if you have more dietary columns
    }


    def find_recipes(selected_ingredients, selected_diets):
        """
        Query Pinecone index for recipes that best match the user-selected
        ingredients and dietary restrictions. Returns a list of matching recipes
        (with minimal metadata) for display in the UI.

        Parameters:
        -----------
        selected_ingredients : list
            Ingredients chosen by the user, e.g. ["Eggs", "Milk"]
        selected_diets : list
            Dietary restrictions chosen by the user, e.g. ["Vegan", "Nut-Free"]

        Returns:
        --------
        results : list of dict
            Each dict represents a recipe match with fields like:
            {
            'id': 'pinecone_vector_id',
            'name': 'Recipe Name',
            'rating': <ratings>,
            'tags': [...],
            ...
            }
            If no matches found, returns an empty list.
        index_value : int
            We'll use this as the "current_index" in Gradio. Typically 0.
        card_text : str
            Text to initially display in the recipe card for the first match.
        suggestions_section_visibility : gr.update
        prev_button_visibility : gr.update
        next_button_visibility : gr.update
        view_button_visibility : gr.update
            Visibility toggles for Gradio components.
        """
        # 1. Build a text query from selected ingredients and diets
        #    (This is optional; you can do a more advanced approach if you want.)
        if selected_ingredients:
            ing_list_str = ", ".join(selected_ingredients)
        else:
            ing_list_str = "any ingredients"

        if selected_diets:
            diets_str = ", ".join(selected_diets)
        else:
            diets_str = "no specific restrictions"

        user_query = f"Recipes that use {ing_list_str} with {diets_str}"

        # 2. Convert the query text into an embedding
        #    (Using your get_recipe_embedding from dataset.py)
        query_embedding = get_recipe_embedding(user_query)

        # 3. Construct a Pinecone filter from the selected diets
        #    For each diet selected, we match it to the corresponding True/False metadata field
        pinecone_filter = {}
        for diet in selected_diets:
            if diet in diet_flags_mapping:
                pinecone_filter[diet_flags_mapping[diet]] = True

        # 4. Query the Pinecone index
        #    Adjust top_k as needed (default 5 or 10, etc.)
        top_k = 5
        if pinecone_filter:
            query_results = index.query(
                vector=query_embedding,
                top_k=top_k,
                include_metadata=True,
                filter=pinecone_filter
            )
        else:
            # If no dietary restrictions, omit filter entirely
            query_results = index.query(
                vector=query_embedding,
                top_k=top_k,
                include_metadata=True
            )

        matches = query_results.get('matches', [])

        # 5. Parse matches into a list of recipe dicts
        if not matches:
            # If no matches found, return an empty array and a "no results" message
            return (
                [],  # suggestions_state
                0,   # current_index
                "*No recipes found. Try different ingredients.*",
                gr.update(visible=True),
                gr.update(visible=False),
                gr.update(visible=False),
                gr.update(visible=False)
            )

        # Convert Pinecone match data into a simpler list of dicts
        results = []
        for match in matches:
            meta = match.get('metadata', {})
            recipe_dict = {
                'id': match['id'],  # Pinecone vector ID
                'name': meta.get('name', 'Unnamed Recipe'),
                'rating': meta.get('ratings', 'N/A'),
                'tags': meta.get('tags', []),
                'score': match.get('score', 0),
                'recipe_id': meta.get('recipe_id'),  # your original DB recipe ID
            }
            results.append(recipe_dict)

        # 6. Prepare to show the first recipe in the UI
        first_recipe = results[0]
        card_text = f"**{first_recipe['name']}** (Rating: {first_recipe['rating']} ⭐)"

        # Return results + gradio updates for displaying
        return (
            results,        # suggestions_state
            0,              # current_index
            card_text,      # initial recipe_card text
            gr.update(visible=True),  # show suggestions_section
            gr.update(visible=True),  # show prev_btn
            gr.update(visible=True),  # show next_btn
            gr.update(visible=True)   # show view_btn
        )

    # Attach find_recipes to the search button
    find_button.click(
        fn=find_recipes,
        inputs=[ingredient_selector, diet_selector],
        outputs=[suggestions_state, current_index, recipe_card,
                 suggestions_section, prev_btn, next_btn, view_btn]
    )

    # Navigation functions for suggestions
    def show_next(idx, suggestions):
        if idx < len(suggestions) - 1:
            idx += 1
        recipe = suggestions[idx]
        card_text = f"**{recipe['name']}** (Rating: {recipe.get('rating', 'N/A')}⭐)"
        return idx, card_text

    def show_prev(idx, suggestions):
        if idx > 0:
            idx -= 1
        recipe = suggestions[idx]
        card_text = f"**{recipe['name']}** (Rating: {recipe.get('rating', 'N/A')}⭐)"
        return idx, card_text

    # Attach navigation functions to buttons
    next_btn.click(show_next, [current_index, suggestions_state], [current_index, recipe_card])
    prev_btn.click(show_prev, [current_index, suggestions_state], [current_index, recipe_card])

    # Function to load detailed info for the selected recipe
    def load_details(idx, suggestions):
        recipe_id = suggestions[idx]["id"]
        # Fetch the full details from Firestore using recipe_id:
        # doc = db.collection("recipes").document(recipe_id).get()
        # data = doc.to_dict() if doc.exists else {}
        # For demo, we'll mock the data:
        data = {
            "name": suggestions[idx]["name"],
            "prep_time": "30 minutes",
            "rating": suggestions[idx].get("rating", "N/A"),
            "ingredients": ["Ingredient A", "Ingredient B", "Ingredient C"],
            "steps": ["Step 1 details...", "Step 2 details...", "Step 3 details..."]
        }
        # Format details into Markdown
        details_text = f"### {data['name']}\n"
        details_text += f"**Preparation Time:** {data['prep_time']}\n\n"
        details_text += f"**Rating:** {data['rating']}⭐\n\n"
        details_text += "**Ingredients:**\n"
        for ing in data["ingredients"]:
            details_text += f"- {ing}\n"
        details_text += "\n**Preparation Steps:**\n"
        for i, step in enumerate(data["steps"], start=1):
            details_text += f"{i}. {step}\n"
        # Switch to detail view (hide suggestions, show details)
        return details_text, gr.update(visible=False), gr.update(visible=True)

    # Attach load_details to the View Details button
    view_btn.click(load_details, [current_index, suggestions_state], [recipe_details_md, suggestions_section, detail_section])

    # Attach Back button to return to suggestions
    back_btn.click(lambda: (gr.update(visible=False), gr.update(visible=True)),
                   inputs=None, outputs=[detail_section, suggestions_section])

In [ ]:
app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://07c4b28d9f5816c658.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
